### Import

In [1]:
import os
import json
import openai
import pandas as pd
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
#from langchain import PromptTemplate, LLMChain
from langchain.prompts import ChatPromptTemplate

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv('MY_OPENAI')

In [3]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser

### Loading Referecne Data

In [4]:
df_two = pd.read_csv("KnowledgeTransfers-two_sent.csv")

In [5]:
### Selecting all annotated cases of family relations
df_fam_two = df_two[df_two.relation.isin(["parent_of", "child_of", "siblings", "spouses"])].copy(deep=True)
df_fam_two.reset_index(drop=True, inplace=True)

In [6]:
df_fam_two.drop([11, 19, 20, 22], inplace=True)
df_fam_two.reset_index(drop=True, inplace=True)

In [7]:
df_fam_two.shape

(85, 11)

In [8]:
df_test = df_fam_two[:14].copy(deep=True)

### Prompt Template and Inference Function

In [9]:
temp_info = """
You are a literary scholar. 
What kind of family relationship between {person_1} and {person_2} is conveyed in the following german piece of text.
###
German piece of text: {drama_snippet}
###
Choose one of "parent_of", child_of", "siblings", "spouses".
JUST name the label without quotation marks and nothing else!
Family relation:
"""

In [10]:
def chaining(person_1, person_2, snippet):
    prompt = ChatPromptTemplate.from_template(template=temp_info)
    messages = prompt.format_messages(person_1=person_1, person_2=person_2, drama_snippet=snippet 
                                #format_instructions=formated
                                     )
    chat = ChatOpenAI(temperature=0.0, model="gpt-4")
    response = chat(messages)
    #print(response.content)
    data = response.content
    return data

### Inference

In [11]:
results = []
for index, row in df_fam_two.iterrows():
    person_1 = row["arg1"]
    person_2 = row["arg2"]
    snippet = row["context"]
    temp_info
    data = chaining(person_1, person_2, snippet)
    results.append(data)

child_of
child_of
child_of
siblings
siblings
siblings
siblings
siblings
siblings
parent_of
parent_of
spouses
spouses
parent_of
spouses
siblings
siblings
parent_of
parent_of
parent_of
siblings
siblings
parent_of
spouses
siblings
siblings
parent_of
parent_of
parent_of
siblings
siblings
parent_of
parent_of
siblings
siblings
child_of
siblings
spouses
child_of
siblings
parent_of
spouses
parent_of
parent_of
child_of
parent_of
parent_of
parent_of
parent_of
parent_of
parent_of
child_of
parent_of
child_of
parent_of
child_of
siblings
siblings
spouses
parent_of
siblings
siblings
siblings
siblings
spouses
parent_of
child_of
spouses
The text does not provide information about the family relationship between Karl and Lucie.
parent_of
siblings
siblings
parent_of
child_of
parent_of
parent_of
parent_of
parent_of
siblings
child_of
siblings
child_of
spouses
parent_of
parent_of


### Evaluation

In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [13]:
results_se = pd.Series(results)

In [14]:
ground_truth = df_fam_two.relation
predictions = results_se

In [15]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,F1,Recall,Precision,Accuracy
0,0.520939,0.552941,0.507624,0.552941
